# Geogebra look-a-like
Here I have created a geogebra lookalike for Seamus!

In [11]:
from IPython.display import HTML


html_content="""<div class="instructions">
        <h2>Triangle Angle Sum Demonstration</h2>
        <p>Drag any vertex of the triangle to change its shape. Notice that the sum of the internal angles always remains 180°.</p>
    </div>
    <div id="box"></div>
    <div class="angle-display">
        <div>Angle A: <span id="angleA">0°</span></div>
        <div>Angle B: <span id="angleB">0°</span></div>
        <div>Angle C: <span id="angleC">0°</span></div>
    </div>
    <div class="angle-sum">Sum of angles: <span id="sumAngles">180°</span></div>
    <div class="angle-sum">Area of triangle: <span id="triangleArea">0</span> square units</div>

    <script>
        // Initialize JSXGraph board
        const board = JXG.JSXGraph.initBoard('box', {
            boundingbox: [-5, 5, 5, -5],
            axis: true,
            grid: true,
            showCopyright: false,
            showNavigation: true
        });

        // Create coordinate texts for each vertex
        function createCoordinateText(point, offsetX, offsetY) {
            return board.create('text', [
                function() { return point.X() + offsetX; },
                function() { return point.Y() + offsetY; },
                function() { return '(' + point.X().toFixed(2) + ', ' + point.Y().toFixed(2) + ')'; }
            ], {fontSize: 12});
        }
        
        // Create triangle vertices
        const A = board.create('point', [0, 0], { 
            name: 'A', 
            size: 5, 
            color: 'red',
            label: { offset: [-10, -10] }
        });
        const B = board.create('point', [4, 0], { 
            name: 'B', 
            size: 5, 
            color: 'green',
            label: { offset: [10, -10] }
        });
        const C = board.create('point', [2, 3], { 
            name: 'C', 
            size: 5, 
            color: 'blue',
            label: { offset: [10, 10] }
        });
        
        // Add coordinate labels
        const coordA = createCoordinateText(A, 0, -0.4);
        const coordB = createCoordinateText(B, 0, -0.4);
        const coordC = createCoordinateText(C, 0, 0.4);

        // Create triangle sides with length labels
        const a = board.create('segment', [B, C], { 
            name: 'a', 
            withLabel: true,
            label: { position: 'top' }
        });
        const b = board.create('segment', [A, C], { 
            name: 'b', 
            withLabel: true,
            label: { position: 'top' }
        });
        const c = board.create('segment', [A, B], { 
            name: 'c', 
            withLabel: true,
            label: { position: 'top' }
        });
        
        // Length calculations for sides
        const lengthA = board.create('text', [
            function() { return (B.X() + C.X())/2 + 0.3; }, 
            function() { return (B.Y() + C.Y())/2 + 0.3; },
            function() { return Math.sqrt(Math.pow(B.X() - C.X(), 2) + Math.pow(B.Y() - C.Y(), 2)).toFixed(2); }
        ], {fontSize: 12});
        
        const lengthB = board.create('text', [
            function() { return (A.X() + C.X())/2 - 0.5; }, 
            function() { return (A.Y() + C.Y())/2 + 0.2; },
            function() { return Math.sqrt(Math.pow(A.X() - C.X(), 2) + Math.pow(A.Y() - C.Y(), 2)).toFixed(2); }
        ], {fontSize: 12});
        
        const lengthC = board.create('text', [
            function() { return (A.X() + B.X())/2; }, 
            function() { return (A.Y() + B.Y())/2 - 0.3; },
            function() { return Math.sqrt(Math.pow(A.X() - B.X(), 2) + Math.pow(A.Y() - B.Y(), 2)).toFixed(2); }
        ], {fontSize: 12});

        // Create angles
        const angleA = board.create('angle', [B, A, C], { 
            radius: 0.5, 
            name: '&alpha;', 
            fillColor: 'rgba(255,0,0,0.2)',
            strokeColor: 'red'
        });
        
        const angleB = board.create('angle', [C, B, A], { 
            radius: 0.5, 
            name: '&beta;', 
            fillColor: 'rgba(0,255,0,0.2)',
            strokeColor: 'green'
        });
        
        const angleC = board.create('angle', [A, C, B], { 
            radius: 0.5, 
            name: '&gamma;', 
            fillColor: 'rgba(0,0,255,0.2)',
            strokeColor: 'blue'
        });

        // Function to convert radians to degrees
        function toDegrees(radians) {
            return radians * 180 / Math.PI;
        }

        // Function to update angle displays and area
        function updateAngles() {
            const angleAValue = toDegrees(angleA.Value()).toFixed(1);
            const angleBValue = toDegrees(angleB.Value()).toFixed(1);
            const angleCValue = toDegrees(angleC.Value()).toFixed(1);
            
            document.getElementById('angleA').textContent = angleAValue + '°';
            document.getElementById('angleB').textContent = angleBValue + '°';
            document.getElementById('angleC').textContent = angleCValue + '°';
            
            const sum = (parseFloat(angleAValue) + parseFloat(angleBValue) + parseFloat(angleCValue)).toFixed(1);
            document.getElementById('sumAngles').textContent = sum + '°';
            
            // Calculate and update area
            // Using the formula: Area = 0.5 * |x1(y2 - y3) + x2(y3 - y1) + x3(y1 - y2)|
            const area = 0.5 * Math.abs(
                A.X() * (B.Y() - C.Y()) + 
                B.X() * (C.Y() - A.Y()) + 
                C.X() * (A.Y() - B.Y())
            );
            document.getElementById('triangleArea').textContent = area.toFixed(2);
        }

        // Update angles on any change
        board.on('update', function() {
            updateAngles();
        });

        // Initial update
        updateAngles();
    </script>"""

HTML(html_content)